In [1]:
import pandas as pd
import yfinance as yf
import datetime

## MSCI: MSCI, BlackRock: 'BLK', S&P500:'^GSPC'

In [2]:
help(yf.Ticker.info)

Help on property:




In [3]:
start, end = "2021-01-01", "2021-07-31"
tickers = ['MSCI', 'BLK', '^GSPC']

In [4]:
df = yf.download("MSCI BLK ^GSPC", start=start, end=end, group_by="ticker")
# yf.download function does not keep the order, below line resorts the top level col indices (tickers)
df = df[tickers]
display(df.head(3))
display(df.tail(3))

[*********************100%***********************]  3 of 3 completed


MSCI                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2020-12-31  437.450012  446.899994  436.359985  446.529999  444.981689   
2021-01-04  450.760010  455.029999  433.410004  437.640015  436.122528   
2021-01-05  436.839996  440.089996  426.450012  431.369995  429.874268   

                           BLK                                      \
            Volume        Open        High         Low       Close   
Date                                                                 
2020-12-31  255600  707.479980  722.419983  706.919983  721.539978   
2021-01-04  408600  730.599976  730.599976  702.039978  710.820007   
2021-01-05  319400  708.809998  717.210022  705.260010  714.580017   

                                      ^GSPC                            \
             Adj Close  Volume         Open         High          Low   
Date                                                                    
2020-12-31  713.953735  361500  3733.270020  3760.199951  3726.879883   
2021-01-04  703.346436  623600  3764.610107  3769.989990  3662.709961   
2021-01-05  707.066895  431400  3698.020020  3737.830078  3695.070068   

                                                  
                  Close    Adj Close      Volume  
Date                                              
2020-12-31  3756.070068  3756.070068  3172510000  
2021-01-04  3700.649902  3700.649902  5006680000  
2021-01-05  3726.860107  3726.860107  4582620000

MSCI                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2021-07-28  588.909973  593.859985  586.270020  591.840027  591.840027   
2021-07-29  592.000000  597.530029  589.260010  594.760010  594.760010   
2021-07-30  592.369995  601.690002  591.840027  595.960022  595.960022   

                           BLK                                      \
            Volume        Open        High         Low       Close   
Date                                                                 
2021-07-28  261000  861.619995  864.169983  854.119995  860.710022   
2021-07-29  260700  866.140015  872.359985  856.760010  869.390015   
2021-07-30  279100  868.250000  875.210022  863.989990  867.169983   

                                      ^GSPC                            \
             Adj Close  Volume         Open         High          Low   
Date                                                                    
2021-07-28  860.710022  389200  4402.950195  4415.470215  4387.009766   
2021-07-29  869.390015  335200  4403.589844  4429.970215  4403.589844   
2021-07-30  867.169983  503700  4395.120117  4412.250000  4389.649902   

                                                  
                  Close    Adj Close      Volume  
Date                                              
2021-07-28  4400.640137  4400.640137  3215130000  
2021-07-29  4419.149902  4419.149902  2815510000  
2021-07-30  4395.259766  4395.259766  2861600000

In [5]:
df.columns.values

array([('MSCI', 'Open'), ('MSCI', 'High'), ('MSCI', 'Low'),
       ('MSCI', 'Close'), ('MSCI', 'Adj Close'), ('MSCI', 'Volume'),
       ('BLK', 'Open'), ('BLK', 'High'), ('BLK', 'Low'), ('BLK', 'Close'),
       ('BLK', 'Adj Close'), ('BLK', 'Volume'), ('^GSPC', 'Open'),
       ('^GSPC', 'High'), ('^GSPC', 'Low'), ('^GSPC', 'Close'),
       ('^GSPC', 'Adj Close'), ('^GSPC', 'Volume')], dtype=object)

In [6]:
msft = yf.Ticker("MSFT")
import pprint
pprint.pprint(msft.info.keys())

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'state', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'industry', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'f

In [7]:
eth = yf.Ticker("ETH-USD")

# OTP Bank Plc. is the largest bank in Hungary with presence in several countries
otp = yf.Ticker("OTP.BD")

#### Get stock time-series data

In [8]:
stock_price = msft.history(period="max")
stock_price.index = pd.to_datetime(stock_price.index)

display(stock_price.head(3))
display(stock_price.tail(3))

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13,0.056108,0.064359,0.056108,0.061608,1031788800,0.0,0.0
1986-03-14,0.061608,0.064909,0.061608,0.063809,308160000,0.0,0.0
1986-03-17,0.063809,0.065459,0.063809,0.064909,133171200,0.0,0.0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-07-30,285.170013,286.660004,283.910004,284.910004,20940900,0.0,0.0
2021-08-02,286.359985,286.769989,283.739990,284.820007,16253900,0.0,0.0
2021-08-03,285.420013,287.230011,284.000000,287.119995,17433975,0.0,0.0


In [9]:
for k, v in msft.info.items():
    print(f"Key: {k} --- Value: {v}")

Key: zip --- Value: 98052-6399
Key: sector --- Value: Technology
Key: fullTimeEmployees --- Value: 181000
Key: longBusinessSummary --- Value: Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, organizations, and enterprise divisions. Its Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and manag

In [10]:
keep = [
    'sector', 'fullTimeEmployees', 'state', 'country', 'website', 'maxAge', 'industry',
    'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins',
    'ebitda', 'recommendationKey','grossProfits', 'freeCashflow', 'currentPrice', 'currentRatio', 'returnOnAssets',
    'averageDailyVolume10Day', 'regularMarketPreviousClose', 'fiftyDayAverage', 'open', 'averageVolume10days',
    'regularMarketDayLow', 'averageVolume', 'dayLow', 'ask', 'askSize', 'volume', 'bid', 'bidSize',
    'dayHigh', 'regularMarketPrice', 'logo_url'
]

In [11]:
reference_data = {k: [v] for k, v in msft.info.items() if k in keep}

In [12]:
reference_data

{'sector': ['Technology'],
 'fullTimeEmployees': [181000],
 'state': ['WA'],
 'country': ['United States'],
 'website': ['http://www.microsoft.com'],
 'maxAge': [1],
 'industry': ['Software—Infrastructure'],
 'ebitdaMargins': [0.48080003],
 'profitMargins': [0.36451998],
 'grossMargins': [0.68926],
 'operatingCashflow': [76740001792],
 'revenueGrowth': [0.213],
 'operatingMargins': [0.41595],
 'ebitda': [80815996928],
 'recommendationKey': ['buy'],
 'grossProfits': [115856000000],
 'freeCashflow': [41337249792],
 'currentPrice': [287.12],
 'currentRatio': [2.08],
 'returnOnAssets': [0.13761],
 'averageDailyVolume10Day': [24285033],
 'regularMarketPreviousClose': [284.82],
 'fiftyDayAverage': [275.09915],
 'open': [285.42],
 'averageVolume10days': [24285033],
 'regularMarketDayLow': [284],
 'averageVolume': [24102467],
 'dayLow': [284],
 'ask': [287.09],
 'askSize': [1100],
 'volume': [17433975],
 'bid': [287.05],
 'bidSize': [1800],
 'dayHigh': [287.23],
 'regularMarketPrice': [287.12]

In [13]:
today = datetime.date.today().strftime('%Y-%m-%d')
reference_data['date'] = today

In [14]:
reference_data

{'sector': ['Technology'],
 'fullTimeEmployees': [181000],
 'state': ['WA'],
 'country': ['United States'],
 'website': ['http://www.microsoft.com'],
 'maxAge': [1],
 'industry': ['Software—Infrastructure'],
 'ebitdaMargins': [0.48080003],
 'profitMargins': [0.36451998],
 'grossMargins': [0.68926],
 'operatingCashflow': [76740001792],
 'revenueGrowth': [0.213],
 'operatingMargins': [0.41595],
 'ebitda': [80815996928],
 'recommendationKey': ['buy'],
 'grossProfits': [115856000000],
 'freeCashflow': [41337249792],
 'currentPrice': [287.12],
 'currentRatio': [2.08],
 'returnOnAssets': [0.13761],
 'averageDailyVolume10Day': [24285033],
 'regularMarketPreviousClose': [284.82],
 'fiftyDayAverage': [275.09915],
 'open': [285.42],
 'averageVolume10days': [24285033],
 'regularMarketDayLow': [284],
 'averageVolume': [24102467],
 'dayLow': [284],
 'ask': [287.09],
 'askSize': [1100],
 'volume': [17433975],
 'bid': [287.05],
 'bidSize': [1800],
 'dayHigh': [287.23],
 'regularMarketPrice': [287.12]

In [27]:
ref_df_today = pd.DataFrame(reference_data)
ref_df_today.set_index('date', inplace=True)
ref_df_today.head()

,sector,fullTimeEmployees,state,country,website,maxAge,industry,ebitdaMargins,profitMargins,grossMargins,...,averageVolume,dayLow,ask,askSize,volume,bid,bidSize,dayHigh,regularMarketPrice,logo_url
date,,,,,,,,,,,,,,,,,,,,,
2021-08-03,Technology,181000,WA,United States,http://www.microsoft.com,1,Software—Infrastructure,0.4808,0.36452,0.68926,...,24102467,284,287.09,1100,17433975,287.05,1800,287.23,287.12,https://logo.clearbit.com/microsoft.com


In [35]:
ref_csv_file = "ref_df.csv"

In [39]:
ref_df_today.to_csv(ref_csv_file, mode='a', header=False)

In [40]:
ref_df = pd.read_csv(ref_csv_file, names = list(ref_df_today.columns))

In [41]:
ref_df.head()

,sector,fullTimeEmployees,state,country,website,maxAge,industry,ebitdaMargins,profitMargins,grossMargins,...,averageVolume,dayLow,ask,askSize,volume,bid,bidSize,dayHigh,regularMarketPrice,logo_url
2021-08-03,Technology,181000,WA,United States,http://www.microsoft.com,1,Software—Infrastructure,0.4808,0.36452,0.68926,...,24102467,284,287.09,1100,17433975,287.05,1800,287.23,287.12,https://logo.clearbit.com/microsoft.com
2021-08-03,Technology,181000,WA,United States,http://www.microsoft.com,1,Software—Infrastructure,0.4808,0.36452,0.68926,...,24102467,284,287.09,1100,17433975,287.05,1800,287.23,287.12,https://logo.clearbit.com/microsoft.com


In [ ]:
with open(csv_file, "a", encoding='utf-8') as f:
    f.write(str(reference_data))

In [ ]:
with open(text_file, "r") as f:
    print(f.read())

In [ ]:
file = dict(f)

In [ ]:
#drop not needed cols
stock_price = stock_price.iloc[:,0:4]
stock_price['zip'] = msft.info['zip']
display(stock_price.head())

#### Get reference data (the file intentionally contains errors)

In [ ]:
ref_data = pd.read_csv('reference_data.csv', index_col='date', parse_dates=True)
display(ref_data)

#### -Task1: Display all columns form ref_data and stock_price in one table. Drop prices where ref_data is not available.

In [ ]:
task1_df = pd.merge(stock_price, ref_data, left_index=True, right_index=True, how='inner')
display(task1_df)

#### -Task2: Convert the ref_data DataFrame into a historical format (valid_from, valid_to values) to capture changes in the time-series data

In [ ]:
cols = ['valid_from', 'valid_to', 'zip'] + list(ref_data.columns)
historical_df_structure = pd.DataFrame(columns=cols)

In [ ]:
historical_df_structure